# Data Modeling for US Accidents Dataset

I have utilized dimensional modeling to structure the US Accidents dataset, employing the star schema to construct both Fact and Dimension Tables.

Fact Table 
* `FactAccidents` - FactAccidents.csv

Dimension Tables
* `DimWeather` - DimWeather.csv
* `DimTimeofDay` - DimTimeofDay.csv
* `DimLocation` - DimLocation.csv
* `DimBinaryFlag` - DimBinaryFlag.csv
* `DimDate` - Use PowerQuery 

In [6]:
# import necessary libaries/packages
import pandas as pd
import numpy as np
import datetime
from datetime import date

In [31]:
# loading data 
df = pd.read_csv(r"C:\Users\jzqma\OneDrive\Documents\Jason's Stuff\Portfolio Projects\US Accidents\US_Accidents_March23.csv")

df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


In [32]:
# Convert the 'Start_Time' column to datetime format using pandas
df["Start_Time"] = pd.to_datetime(df["Start_Time"])

# Extract the date from the 'Start_Time' column and store it in a new 'Date' column
df["Date"] = df['Start_Time'].dt.date

In [33]:
df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Date
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,Night,Night,Night,Night,2016-02-08
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,Night,Night,Night,Day,2016-02-08
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,True,False,Night,Night,Day,Day,2016-02-08
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,Night,Day,Day,Day,2016-02-08
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,True,False,Day,Day,Day,Day,2016-02-08


In [35]:
# Get the column names of the DataFrame and store them in df_cols
df_cols = df.columns

# Print the number of columns in the DataFrame using an f-string
print(f"The Number of Columns: {len(df_cols)}")

The Number of Columns: 47


In [36]:
# Define columns for the FactAccidents table
fact_cols =  ['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 
                 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)', 'Description']

# Define columns for the DimLocation table
dim_location_cols = ['Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code']

# Define columns for the DimWeather table
dim_weather_cols = ['Wind_Direction', 'Weather_Condition']

# Define columns for the DimBinaryFlags table
dim_binary_cols = ['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']

# Define columns for the DimTimeOfDay table
dim_timeofday_cols = ['Sunrise_Sunset','Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']

# Creating Fact and Dim Tables

## Creating Fact Table

In [37]:
# Select specific columns from the DataFrame to create the fact table
fact_table = df[['ID', 'Source', 'Severity', 'Start_Time', 'Date', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 
                 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)', 'Description']]

## Creating Dim Tables

In [38]:
# Select specific columns from the DataFrame to create the dim_location table
dim_location = df[['Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code']]

In [39]:
dim_location.head()

,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code
0,I-70 E,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO
1,Brice Rd,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH
2,State Route 32,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69
3,I-75 S,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY
4,Miamisburg Centerville Rd,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY


In [40]:
# Select specific columns from the DataFrame to create the dim_weather table
dim_weather = df[['Wind_Direction', 'Weather_Condition']]

In [41]:
dim_weather.head()

,Wind_Direction,Weather_Condition
0,Calm,Light Rain
1,Calm,Light Rain
2,SW,Overcast
3,SW,Mostly Cloudy
4,SW,Mostly Cloudy


In [42]:
# Select specific columns from the DataFrame to create the dim_binary table
dim_binary = df[['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]

In [43]:
dim_binary.head()

,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False


In [44]:
# Select specific columns from the DataFrame to create the dim_time_of_day table
dim_time_of_day = df[['Sunrise_Sunset','Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']]

In [45]:
dim_time_of_day.head()

,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,Night,Night,Night,Night
1,Night,Night,Night,Day
2,Night,Night,Day,Day
3,Night,Day,Day,Day
4,Day,Day,Day,Day


In [47]:
def add_id(df, new_col_name):
    """
    Add a new column to the DataFrame with unique identifiers.

    Parameters:
    - df (DataFrame): The input DataFrame.
    - new_col_name (str): The name of the new column.

    Returns:
    - DataFrame: A copy of the input DataFrame with the new column added.
    """
    
    original_rows = df.shape[0]
    
    # Drop Duplicate Rows
    df = df.drop_duplicates().reset_index(drop=True)
    
    after_drop_dups_rows = df.shape[0]
    print(f"Original Rows: {original_rows}")
    print(f"After Rows: {after_drop_dups_rows}")
    print(f"Total Drop Rows: {original_rows - after_drop_dups_rows}")
    
    # Add locationID 
    df[new_col_name] = np.arange(1, original_rows+1)
    
    return df

In [48]:
# Add unique identifiers to dimension tables
dim_location = add_id(dim_location, "location_id")
dim_weather = add_id(dim_weather, "weather_id")
dim_binary = add_id(dim_binary, "binary_id")
dim_time_of_day = add_id(dim_time_of_day, "time_of_day_id")

Original Rows: 7728394
After Rows: 1261825
Total Drop Rows: 6466569
Original Rows: 7728394
After Rows: 1706
Total Drop Rows: 7726688
Original Rows: 7728394
After Rows: 344
Total Drop Rows: 7728050
Original Rows: 7728394
After Rows: 11
Total Drop Rows: 7728383


## Merging Dim Foreign Keys to Fact Table

In [49]:
# reference: https://stackoverflow.com/questions/17978133/python-pandas-merge-only-certain-columns

# Merge fact table with dim_location and add location_id
fact_table = pd.merge(df, dim_location[['Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', "location_id"]], on=dim_location_cols, how="left")
fact_table = fact_table.drop(dim_location_cols, axis=1)

# Merge fact table with dim_weather and add weather_id
fact_table = pd.merge(fact_table, dim_weather[['Wind_Direction', 'Weather_Condition', 'weather_id']], on=dim_weather_cols, how="left")
fact_table = fact_table.drop(dim_weather_cols, axis=1)

# Merge fact table with dim_binary and add binary_id
fact_table = pd.merge(fact_table, dim_binary[['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'binary_id']], on=dim_binary_cols, how="left")
fact_table = fact_table.drop(dim_binary_cols, axis=1)

# Merge fact table with dim_time_of_day and add time_of_day_id
fact_table = pd.merge(fact_table, dim_time_of_day[['Sunrise_Sunset','Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'time_of_day_id']], on=dim_timeofday_cols, how="left")
fact_table = fact_table.drop(dim_timeofday_cols, axis=1)

In [50]:
fact_table.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Date,location_id,weather_id,binary_id,time_of_day_id
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,91.0,29.68,10.0,NaN,0.02,2016-02-08,1,1,1,1
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,100.0,29.65,10.0,NaN,0.00,2016-02-08,2,1,1,2
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,100.0,29.67,10.0,3.5,NaN,2016-02-08,3,2,2,3
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,96.0,29.64,9.0,4.6,NaN,2016-02-08,4,3,1,4
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,89.0,29.65,6.0,3.5,NaN,2016-02-08,5,3,2,5


In [55]:
# checking column names
fact_table.columns

Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Date', 'location_id', 'weather_id', 'binary_id',
       'time_of_day_id'],
      dtype='object')

In [56]:
fact_table.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Date,location_id,weather_id,binary_id,time_of_day_id
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,91.0,29.68,10.0,NaN,0.02,2016-02-08,1,1,1,1
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,100.0,29.65,10.0,NaN,0.00,2016-02-08,2,1,1,2
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,100.0,29.67,10.0,3.5,NaN,2016-02-08,3,2,2,3
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,96.0,29.64,9.0,4.6,NaN,2016-02-08,4,3,1,4
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,89.0,29.65,6.0,3.5,NaN,2016-02-08,5,3,2,5


# Export to CSV


In [58]:
# Save the fact table to a CSV file
fact_table.to_csv(r"C:\Users\jzqma\OneDrive\Documents\Jason's Stuff\Portfolio Projects\US Accidents\data\FactAccidents.csv", index=False)

# Save the dim_location table to a CSV file
dim_location.to_csv(r"C:\Users\jzqma\OneDrive\Documents\Jason's Stuff\Portfolio Projects\US Accidents\data\DimLocation.csv", index=False)

# Save the dim_weather table to a CSV file
dim_weather.to_csv(r"C:\Users\jzqma\OneDrive\Documents\Jason's Stuff\Portfolio Projects\US Accidents\data\DimWeather.csv", index=False)

# Save the dim_binary table to a CSV file
dim_binary.to_csv(r"C:\Users\jzqma\OneDrive\Documents\Jason's Stuff\Portfolio Projects\US Accidents\data\DimBinaryFlag.csv", index=False)

# Save the dim_time_of_day table to a CSV file
dim_time_of_day.to_csv(r"C:\Users\jzqma\OneDrive\Documents\Jason's Stuff\Portfolio Projects\US Accidents\data\DimTimeofDay.csv", index=False)

In [7]:
current_date = date.today()
print(f"Today's date: {current_date}")
print("Report Complete")
print("")

Today's date: 2024-02-28
Report Complete

